In [22]:
import re
import h5py
import numpy as np
import scipy as sp
from numba import jit
import matplotlib.pyplot as plt
from pyXSteam.XSteam import XSteam
#from scipy.interpolate import interp1d
#from scipy.sparse import coo_matrix


In [23]:
"""
The 'sigmaZeros()' function calculates the background cross sections, 
sigma-zeros, based on given input parameters. 

**Inputs:**
- 'sigTtab': A cell array containing matrices of total microscopic 
    cross sections for each isotope. Each matrix has dimensions 
    (nsigz x ng), where nsigz is the number of base points 
    sigma-zeros and ng is the number of energy groups.
- 'sig0tab': A cell array containing vectors of base points of 
    tabulated sigma-zeros for each energy group.
- 'aDen': A vector of atomic densities of isotopes.
- 'SigEscape': The escape cross section (1/cm) for simple convex objects.

**Outputs:**
- 'sig0': A 2D matrix of sigma-zeros with dimensions (nIso x ng).
- 'sigT': A 2D matrix of total macroscopic cross sections corrected 
    with account for sigma-zero, with dimensions (nIso x ng).

The function uses the input parameters to calculate the sigma-zeros and 
the corrected total macroscopic cross sections.
------------------------------------------------------------------------
More on sigma0:
In the context of radiation transport and cross-section data, "sigma-zero" 
refers to the base point or reference value of the cross section at a 
specific energy group. It represents the background or base cross section 
for a given interaction process between particles and materials.

The sigma-zero value is used as a reference point for interpolating or 
extrapolating the cross section values at different energies or sigma-zero 
points. By knowing the sigma-zero value and the variation of the cross 
section with respect to sigma-zero, one can calculate the cross section at 
any desired energy or sigma-zero value.

In practical terms, sigma-zero allows for the normalization and adjustment 
of cross-section data to a common reference point. It helps characterize 
the behavior of cross sections as a function of energy and provides a 
consistent framework for comparing and analyzing different materials 
and isotopes.

By accounting for sigma-zero, one can accurately determine the total 
macroscopic cross sections, which play a crucial role in calculations 
related to radiation shielding, dose calculations, reactor design, and 
other applications in nuclear physics and radiation science.
"""
def sigmaZeros(sigTtab, sig0tab, aDen, SigEscape):
    # Number of energy groups
    ng = 421

    # Define number of isotopes in the mixture
    nIso = len(aDen)

    # Define the size of sigT and a temporary value 
    # named sigT_tmp for interpolation values
    sigT = np.zeros((nIso, ng))
    sigT_tmp = 0

    # first guess for sigma-zeros is 1e10 (infinite dilution)
    sig0 = np.ones((nIso, ng)) * 1e10

    # Loop over energy group
    for ig in range(ng):
        # Error to control sigma-zero iterations
        err = 1e10
        nIter = 0

        # sigma-sero iterations until the error is below selected tolerance (1e-6)
        while err > 1e-6:
            # Loop over isotopes
            for iIso in range(nIso):
                # Find cross section for the current sigma-zero by interpolating
                # in the table
                if np.count_nonzero(sig0tab[iIso]) == 1:
                    sigT[iIso, ig] = sigTtab[iIso][0, ig]
                else:
                    log10sig0 = np.minimum(10, np.maximum(0, np.log10(sig0[iIso, ig])))
                    sigT_tmp = sp.interpolate.interp1d( np.log10(sig0tab[iIso][np.nonzero(sig0tab[iIso])]), 
                                                    sigTtab[iIso][:, ig][np.nonzero(sigTtab[iIso][:, ig])], 
                                                    kind='linear')(log10sig0)
                    sigT[iIso, ig] = sigT_tmp
                #sigT = sigT.item() 
                #sigTtab[iIso][np.isnan(sigTtab[iIso])] = 0  # not sure if 100% necessary, but a good mental check

            err = 0
            # Loop over isotopes
            for iIso in range(nIso):
                # Find the total macroscopic cross section for the mixture of
                # the background isotopes
                summation = 0
                # Loop over background isotopes
                for jIso in range(nIso):
                    if jIso != iIso:
                        summation += sigT[jIso, ig] * aDen[jIso]

                tmp = (SigEscape + summation) / aDen[iIso]
                err += (1 - tmp / sig0[iIso, ig])**2
                sig0[iIso, ig] = tmp

            err = np.sqrt(err)
            nIter += 1
            if nIter > 100:
                print('Error: too many sigma-zero iterations.')
                return

    return sig0

In [24]:
def prepareInto3D(sigA_H01, sigA_O16, sigA_U235):
    data2D = np.concatenate([sigA_H01, sigA_O16, sigA_U235], axis=0)
    
    # Reshape the sigTtab from 2D into a 3D array
    num_cells = 3
    num_rows_per_cell = [sigA_H01.shape[0], sigA_O16.shape[0], sigA_U235.shape[0]]  # Number of rows per cell; 1, 6, 10
    num_cols = data2D.shape[1]  # Number of columns

    # Create an empty 3D array with the desired shape
    result3D = np.zeros((num_cells, max(num_rows_per_cell), num_cols))

    # Fill the 3D array with data from the 2D matrix
    row_start = 0
    for cell in range(num_cells):
        num_rows = num_rows_per_cell[cell]
        result3D[cell, :num_rows, :] = data2D[row_start:row_start+num_rows, :]
        row_start += num_rows

    return result3D

In [63]:
# number of energy groups
H2OU_ng = 421

# Path to microscopic cross section data:
micro_XS_path = '../01.Micro_Python'

# Call the functions for H2O and B isotopes and store the data in the
# structures. As an example it is done below for temperature of 294K,
# pressure of 7 MPa and boron concentration of 760 ppm.
# Change when other parameters needed.
# Open the HDF5 files
hdf5_H01  = h5py.File(micro_XS_path + '/micro_H_001__294K.h5', 'r')
hdf5_O16  = h5py.File(micro_XS_path + '/micro_O_016__294K.h5', 'r')
hdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')

# For looking inside the HDF5 file groups
element = 'O16'
group = 'sigS_G'
subgroup = 'sigS(0,0)'
print(f'Metadata for {element}:', list(eval(f'hdf5_{element}').attrs.keys()))
baseItems = list(eval(f'hdf5_{element}').items())
print(f'Base Items in {element}: ',baseItems)
#eg_G_H01 = hdf5_H01.get('en_G')
groupItems = list(hdf5_O16.get(group).items())
print(f'Items in {group}_{element}:', groupItems)
subgroupData = np.array(hdf5_O16.get(group).get(subgroup))
print(f'NumPy data in {subgroup}:\n',subgroupData[0:5])

H2OU_temp = 294  # K
H2OU_p = 0.1  # MPa
H2OU_Uconc = 1000e-6  # 1e-6 = 1 ppm
#H2OU_eg = H01['eg']
H2OU_eg = np.array(hdf5_H01.get('en_G').get('eg'))   # From the group 'en_G' get subgroup containing the data named 'eg'

# Get the atomic weight from the metadata
H01_aw  = hdf5_H01.attrs.get('atomic_weight_amu')
O16_aw  = hdf5_O16.attrs.get('atomic_weight_amu')
U235_aw = hdf5_U235.attrs.get('atomic_weight_amu')
print('\natomic_weight of H01: ', H01_aw,'\ndata type:', type(H01_aw))

# Mass of one "average" H2OU molecule in atomic unit mass [a.u.m.]:
H2OU_aw = 2 * H01_aw + O16_aw + H2OU_Uconc * U235_aw

# Path to steam-water properties:
#XSteam_path = '../00.pyXSteam'

# The function returns water density at specified pressure (MPa) and
# temperature (C):
steamTable = XSteam(XSteam.UNIT_SYSTEM_MKS)
density = steamTable.rho_pt(H2OU_p/10, H2OU_temp-273)

# The water density:
H2OU_den = density*1e-3  # [g/cm3]
rho = H2OU_den*1.0e-24  # [g/(barn*cm)]
rho = rho / 1.660538e-24  # [(a.u.m.)/(barn*cm)]
rho = rho / H2OU_aw  # [number of H2O molecules/(barn*cm)]

# The names of fissionable isotopes and oxygen
H2OU_isoName = ['H01', 'O16', 'U235']

# The number densities of isotopes:
H2OU_numDen = np.array([2*rho, rho, rho*H2OU_Uconc])

# I have no idea if this is working correctly, but the
# idea is that if a group, let's say sigT_G, has multible
# subgroups named 'sigT(0,:)', 'sigT(1,:)', ..., 'sigT(9,:)'
# then this function should combine all of them into
# one single NumPy array 
def get_subg_data_as_array(group):
    subgroups_data = []

    def get_data(name, obj):
        if isinstance(obj, h5py.Dataset):
            subgroup_data = np.array(obj)
            subgroups_data.append(subgroup_data)

    group.visititems(get_data)
    return np.array(subgroups_data) # The output is np.array([[data]]) 
                                    # That is useful for np.concatenate()
# end of function                       

# Testing
#sigT_G = hdf5_U235.get('sigT_G')
#sigT_data = get_subg_data_as_array(sigT_G)
#print(sigT_G, sigT_data)

# Get total micrscopic cross-section data 
sigT_H01 = get_subg_data_as_array(hdf5_H01.get('sigT_G'))
sigT_O16 = get_subg_data_as_array(hdf5_O16.get('sigT_G'))
sigT_U235 = get_subg_data_as_array(hdf5_U235.get('sigT_G'))

# Make this below into a function:
# Prepare for sigma-zero iterations:
# Concatenate the arrays along the first axis to get a 3D array
sigTtabInit = np.concatenate([sigT_H01, sigT_O16, sigT_U235], axis=0)

# Reshape the sigTtab from 2D into a 3D array
num_cells = 3
num_rows_per_cell = [sigT_H01.shape[0], sigT_O16.shape[0], sigT_U235.shape[0]]  # Number of rows per cell; 1, 6, 10
num_cols = sigTtabInit.shape[1]  # Number of columns

# Create an empty 3D array with the desired shape
sigTtab = np.zeros((num_cells, max(num_rows_per_cell), num_cols))

# Fill the 3D array with data from the 2D matrix
row_start = 0
for cell in range(num_cells):
    num_rows = num_rows_per_cell[cell]
    sigTtab[cell, :num_rows, :] = sigTtabInit[row_start:row_start+num_rows, :]
    row_start += num_rows

# wtf is sigma zero?
sig0_H01  = np.array(hdf5_H01.get( 'sig0_G').get('sig0'))
sig0_O16  = np.array(hdf5_O16.get( 'sig0_G').get('sig0'))
sig0_U235 = np.array(hdf5_U235.get('sig0_G').get('sig0'))
sig0tab = np.concatenate([sig0_H01, sig0_O16, sig0_U235], axis=0)

# NOTE: currently concatenate puts all the data in single vector
# but in the Matlab code, all data vectors are added separately
# and every cell in the resulting 3D/2D matrix have different lengths
# Determine the length of each sig0_* variable
sig0_sizes = [len(sig0_H01), len(sig0_O16), len(sig0_U235)]
isotope = ["H01", "O16", "U235"]
# Create a 2D array where the number of columns is determined by the 
# length of the sig0_* variables
# Not exactly how it is in MATLAB, but at least each row is
# only as long as the longest row filled with non-zero variables
sig0tab = np.zeros((3, max(sig0_sizes)))    # (3,10)
col_start = 0
for i, size in enumerate(sig0_sizes):
    print("i =", i, "size =", size)
    sig0tab[i, col_start:col_start+size] = eval(f'sig0_{isotope[i]}')
    #col_start += size

# The number densities of isotopes, but in a new variable
aDen = H2OU_numDen

# SigEscape -- escape cross section, for simple convex objects (such as
# plates, spheres, or cylinders) is given by S/(4V), where V and S are the
# volume and surface area of the object, respectively
SigEscape = 0

print('Sigma-zero iterations. ')
H2OU_sig0 = sigmaZeros(sigTtab, sig0tab, aDen, SigEscape)
print('Done.\n')

print("Interpolation of microscopic cross sections for the found sigma-zeros.")
sigC_H01 = get_subg_data_as_array(hdf5_H01.get(  'sigC_G'))
sigC_O16 = get_subg_data_as_array(hdf5_O16.get(  'sigC_G'))
sigC_U235 = get_subg_data_as_array(hdf5_U235.get('sigC_G'))


# Get the atomic weight from the metadata
H01_nSig0  = hdf5_H01.attrs.get('nSig0')
O16_nSig0  = hdf5_O16.attrs.get('nSig0')
U235_nSig0 = hdf5_U235.attrs.get('nSig0')

sigL_H01 = np.zeros((H01_nSig0, H2OU_ng))
sigL_O16 = np.zeros((O16_nSig0, H2OU_ng))
sigL_U235= np.zeros((U235_nSig0, H2OU_ng))

sigF_H01 = get_subg_data_as_array(hdf5_H01.get(  'sigF_G'))
sigF_O16 = get_subg_data_as_array(hdf5_O16.get(  'sigF_G'))
sigF_U235 = get_subg_data_as_array(hdf5_U235.get('sigF_G'))

sigCtab = prepareInto3D(sigC_H01, sigC_O16, sigC_U235)
sigLtab = prepareInto3D(sigL_H01, sigL_O16, sigL_U235)
sigFtab = prepareInto3D(sigF_H01[0], sigF_O16[0], sigF_U235) # I don't know why, I don't know how but sigF_H01 and sigF_O16 are 3D, 
                                                             # but should be 2D...
                                                             # Nevertheless I only need to use their first cell to get the 2D matrix
#print(sigCtab.shape)

sigC = np.zeros((3, H2OU_ng))
sigF = np.zeros((3, H2OU_ng))
sigL = np.zeros((3, H2OU_ng))

#=======================================================================================
# Documentation for "interpolate_data()"
#---------------------------------------------------------------------------------------
# The 'kind='linear'' argument in 'interp1d' specifies the type of interpolation to be 
# performed. In this case, it indicates that linear interpolation should be used. 
# Linear interpolation calculates the values between two known data points as a 
# straight line. 
#
# Regarding 'fill_value='extrapolate'', it is used to enable extrapolation of values 
# outside the range of the given data. By default, 'interp1d' raises an error if you 
# try to interpolate/extrapolate outside the range of the input data. Setting 
# 'fill_value='extrapolate'' allows the function to extrapolate values beyond the 
# given data range.
#
# In the MATLAB code, the line 'log10sig0 = min(10,max(0,log10(H2OU.sig0(iIso,ig))))' 
# clamps the value of 'log10sig0' between 0 and 10. This means that if the value is less 
# than 0, it is set to 0, and if it is greater than 10, it is set to 10. The 
# 'fill_value='extrapolate'' argument in the Python code allows for similar 
# extrapolation behavior when 'log10sig0' falls outside the range of the input data.
#=======================================================================================
def interpolate_data(x, y, xi):
    interp_func = sp.interpolate.interp1d(x, y, kind='linear', fill_value='extrapolate')
    return interp_func(xi)

for ig in range(H2OU_ng):
    # Loop over isotopes
    for iIso in range(3):
        # Find cross sections for the sigma-zero
        if np.count_nonzero(sig0tab[iIso]) == 1:
            sigC[iIso, ig] = sigCtab[iIso][0, ig]
            sigL[iIso, ig] = sigLtab[iIso][0, ig]
            sigF[iIso, ig] = sigFtab[iIso][0, ig]
        else:
            log10sig0 = min(10, max(0, np.log10(H2OU_sig0[iIso, ig])))  #sig0tab[1][np.nonzero(sig0tab[1])]
            arrayLength = len(sig0tab[iIso][np.nonzero(sig0tab[iIso])])
            #sigC[iIso, ig] = interpolate_data(np.log10(sig0tab[iIso][np.nonzero(sig0tab[iIso])]), sigCtab[iIso][:, ig][np.nonzero(sigCtab[iIso][:, ig])], log10sig0)
            sigC[iIso, ig] = interpolate_data(np.log10(sig0tab[iIso][:arrayLength]), sigCtab[iIso][:arrayLength, ig], log10sig0)
            sigL[iIso, ig] = interpolate_data(np.log10(sig0tab[iIso][:arrayLength]), sigLtab[iIso][:arrayLength, ig], log10sig0) # May need fixing but work at the moment
            sigF[iIso, ig] = interpolate_data(np.log10(sig0tab[iIso][:arrayLength]), sigFtab[iIso][:arrayLength, ig], log10sig0) # May need fixing but work at the moment
            # *Some helpful information for the lines after the 'else' statement:
            #-----
            # You might be wondering why [:arrayLength] is used instead of [:]. This is to solve two separate problems
            # with one stone. The first issue becomes apparent when all elements in a row from sig0tab are used. The
            # 2D matrix sig0tab contains nonzero values in 3 separate rows with 3 separate dimensions: [1, 6, 10].
            # All the empty slots that don't contain any useful values are filled with zeros like so:
            #  [[1.e+10, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
            #   [1.e+10, 1.e+04, 1.e+03, 1.e+02, 1.e+01, 1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00],
            #   [1.e+10, 1.e+04, 3.e+03, 1.e+03, 3.e+02, 1.e+02, 6.e+01, 3.e+01, 1.e+01, 1.e+00]]
            # As it is apparent from the code snippet above that we need to find a log10() of these nonzero values.
            # Obviously we can't take a log10() of the value 0.0, but Python usually handles this with a simple
            # warning. If we want to get rid of the warning in a clever way, we only need to count how many elements
            # in a single array are not equal to zero and then count up only to that value and no more, hence we use
            # ...][:arrayLength]. The second problem becomes apparent now, because sig*tab[] can contain zero elements
            # for some rows and not for others. Meaning we also need to define the correct length of values for 
            # how many elements we take from sig*tab.Since the inputs x and y in the SciPy interp1d() need to 
            # have the same length, it means we only need to take into account only as many elements that are in x, 
            # hence we also use ...][:arrayLength, ig] for sig*tab. 

"""
======================================================================================
Documentation interpSigS() function
--------------------------------------------------------------------------------------
 The interpSigS function performs interpolation to calculate the scattering matrix 
 sigS based on provided input parameters.
 
 **Inputs**:
 - 'jLgn': An integer representing the index of the energy group.
 - 'element': A string specifying the element.
 - 'Sig0': A numpy array representing the sigma-zero values for target points.

 **Outputs:**
 - 'sigS': A numpy array representing the resulting scattering matrix.
======================================================================================
The 'interpSigS()' function calculates and returns the scattering matrix 'sigS'. 
This matrix is obtained through interpolation between scattering matrices ('s_SigS') 
that correspond to specific sigma-zero base points. The interpolation is performed 
for a set of target points represented by the vector 'Sig0', which has a length of 
'ng' (the number of energy groups).

In other words, the function takes as input the base scattering matrices for different 
sigma-zero values. These matrices capture the scattering behavior of a material under 
different conditions. The function then uses these base matrices to estimate the 
scattering behavior at target points specified by 'Sig0'.

By performing interpolation, the function infers the scattering matrix values at the 
target points based on the known scattering matrices for the sigma-zero base points. 
The resulting 'sigS' matrix provides an approximation of the scattering behavior at 
the target points, enabling further analysis or calculations involving the material's 
scattering properties.
======================================================================================
"""
def interpSigS(jLgn, element, Sig0):
    # number of energy groups
    ng = 421
    s_sig0 = np.array(eval(f'hdf5_{element}').get('sig0_G').get('sig0'))

    findSigS = list(eval(f'hdf5_{element}').get(group).items())
    #print(f'Items in hdf5_{element}:', findSigS)
    string = findSigS[-1][0]

    # Extract numbers using regular expression pattern
    pattern = r"sigS\((\d+),(\d+)\)"
    match = re.search(pattern, string)

    if match:
        x_4D = int(match.group(1)) + 1
        y_4D = int(match.group(2)) + 1
        #print(f"i = {x_4D}, j = {y_4D}")
    else:
        print("No match found.")

    # Create the empty 3D numpy array
    s_sigS = np.zeros((x_4D, y_4D, ng, ng))

    # Access the data from the subgroups and store it in the 3D array
    for i in range(x_4D):
        for j in range(y_4D):
            dataset_name = f'sigS({i},{j})'
            s_sigS[i, j] = eval(f'hdf5_{element}')['sigS_G'][dataset_name][:]

    # number of sigma-zeros
    nSig0 = len(s_sig0)

    if nSig0 == 1:
        sigS = s_sigS[jLgn][0]
    else:
        tmp1 = np.zeros((nSig0, sp.sparse.find(s_sigS[jLgn][0])[2].shape[0]))
        for iSig0 in range(nSig0):
            ifrom, ito, tmp1[iSig0,:] = sp.sparse.find(s_sigS[jLgn][iSig0])
        
        # number of non-zeros in a scattering matrix
        nNonZeros = tmp1.shape[1]
        tmp2 = np.zeros(nNonZeros)
        for i in range(nNonZeros):
            log10sig0 = min(10, max(0, np.log10(Sig0[ifrom[i]])))
            # This part is PAAAAAAAAAIIIINFULLY SLOW...  but works ⊙﹏⊙
            #tmp2[i] = sp.interpolate.interp1d(np.log10(s_sig0), tmp1[:,i], kind='linear')(log10sig0)
            #---
            # np.interp() is 5x faster than sp.interpolate.interp1d(), but does 
            # not handle complex-valued data or ndim>1 which I don't use anyway.
            tmp2[i] = np.interp(np.log10(log10sig0), np.log10(s_sig0), tmp1[:, i])

        sigS = sp.sparse.coo_matrix((tmp2, (ifrom, ito)), shape=(ng, ng)).toarray()

    return sigS
#===================================================================================

# Preallocate the array with zeros
sigS = np.zeros((3, 3, 421, 421))
for j in range(3):
    sigS[j, 0, :, :] = interpSigS(j, 'H01',  H2OU_sig0[0, :])
    sigS[j, 1, :, :] = interpSigS(j, 'O16',  H2OU_sig0[1, :])
    sigS[j, 2, :, :] = interpSigS(j, 'U235', H2OU_sig0[2, :])

print(sigS.shape)
print('Done.')

# Macroscopic cross section [1/cm] is microscopic cross section for the 
# molecule [barn] times the number density [number of molecules/(barn*cm)]
H2OU_SigC = np.transpose(sigC) @ aDen
H2OU_SigL = np.transpose(sigL) @ aDen
H2OU_SigF = np.transpose(sigF) @ aDen
U235_nubar = get_subg_data_as_array((hdf5_U235.get('nubar_G')))
#print(U235_nubar)
H2OU_SigP = U235_nubar * sigF[2, :] * aDen[2] 

H2OU_SigS = np.zeros((3, 421, 421))
for j in range(3):
    H2OU_SigS[j] = sigS[j, 0]*aDen[0] + sigS[j, 1]*aDen[1] + sigS[j, 2]*aDen[2]

#H01_sig2  = get_subg_data_as_array((hdf5_H01 .get('sig2_G')))
H01_sig2  = np.array(hdf5_H01.get('sig2_G').get('sig2'))
O16_sig2  = np.array(hdf5_O16.get('sig2_G').get('sig2'))
U235_sig2 = np.array(hdf5_U235.get('sig2_G').get('sig2'))

H2OU_Sig2 = H01_sig2 * aDen[0] + O16_sig2 * aDen[1] + U235_sig2 * aDen[2]
H2OU_SigT = H2OU_SigC + H2OU_SigL + H2OU_SigF + np.sum(H2OU_SigS[0], axis=1) + np.sum(H2OU_Sig2, axis=1)

# Fission spectrum
U235_chi = get_subg_data_as_array((hdf5_U235.get('chi_G')))
H2OU_chi = U235_chi

    
# Close the HDF5 files
hdf5_H01.close()
hdf5_O16.close()
hdf5_U235.close()


Metadata for O16: ['atomic_weight_amu', 'description', 'nSig0', 'ng', 'temperature']
Base Items in O16:  [('chi_G', <HDF5 group "/chi_G" (1 members)>), ('en_G', <HDF5 group "/en_G" (1 members)>), ('nubar_G', <HDF5 group "/nubar_G" (1 members)>), ('sig0_G', <HDF5 group "/sig0_G" (2 members)>), ('sig2_G', <HDF5 group "/sig2_G" (3 members)>), ('sigC_G', <HDF5 group "/sigC_G" (6 members)>), ('sigE_G', <HDF5 group "/sigE_G" (1 members)>), ('sigF_G', <HDF5 group "/sigF_G" (1 members)>), ('sigL_G', <HDF5 group "/sigL_G" (2 members)>), ('sigS_G', <HDF5 group "/sigS_G" (20 members)>), ('sigT_G', <HDF5 group "/sigT_G" (6 members)>)]
Items in sigS_G_O16: [('ifromS', <HDF5 dataset "ifromS": shape (9863,), type "<i4">), ('itoS', <HDF5 dataset "itoS": shape (9863,), type "<i4">), ('sigS(0,0)', <HDF5 dataset "sigS(0,0)": shape (421, 421), type "<f8">), ('sigS(0,1)', <HDF5 dataset "sigS(0,1)": shape (421, 421), type "<f8">), ('sigS(0,2)', <HDF5 dataset "sigS(0,2)": shape (421, 421), type "<f8">), ('si

In [58]:
print(H2OU_SigS.shape)

(3, 421, 421)


In [34]:
"""#def interpSigS(jLgn, s, Sig0):
hdf5_H01  = h5py.File(micro_XS_path + '/micro_H_001__294K.h5', 'r')
hdf5_O16  = h5py.File(micro_XS_path + '/micro_O_016__294K.h5', 'r')
hdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')


jLgn = 0
Sig0 = H2OU_sig0[1, :]

element = 'U235'

# number of energy groups
ng = 421
s_sig0 = np.array(eval(f'hdf5_{element}').get('sig0_G').get('sig0'))

findSigS = list(eval(f'hdf5_{element}').get(group).items())
print(f'Items in hdf5_{element}:', findSigS)
string = findSigS[-1][0]

# Extract numbers using regular expression pattern
pattern = r"sigS\((\d+),(\d+)\)"
match = re.search(pattern, string)

if match:
    x_4D = int(match.group(1)) + 1
    y_4D = int(match.group(2)) + 1
    print(f"i = {x_4D}, j = {y_4D}")
else:
    print("No match found.")

# Create the empty 3D numpy array
s_sigS = np.zeros((x_4D, y_4D, ng, ng))

# Access the data from the subgroups and store it in the 3D array
#s_sigS[0] = hdf5_H01['sigS_G']['sigS(0,0)'][:]
#s_sigS[1] = hdf5_H01['sigS_G']['sigS(1,0)'][:]
#s_sigS[2] = hdf5_H01['sigS_G']['sigS(2,0)'][:]
for i in range(x_4D):
    for j in range(y_4D):
        dataset_name = f'sigS({i},{j})'
        s_sigS[i, j] = eval(f'hdf5_{element}')['sigS_G'][dataset_name][:]

# number of sigma-zeros
nSig0 = len(s_sig0)
#print(s_sigS[0][0].shape)


if nSig0 == 1:
    sigS = s_sigS[jLgn][0]
else:
    tmp1 = np.zeros((nSig0, sp.sparse.find(s_sigS[jLgn][0])[2].shape[0]))
    for iSig0 in range(nSig0):
        ifrom, ito, tmp1[iSig0,:] = sp.sparse.find(s_sigS[jLgn][iSig0])
    
    # number of non-zeros in a scattering matrix
    nNonZeros = tmp1.shape[1]
    tmp2 = np.zeros(nNonZeros)
    for i in range(nNonZeros):
        log10sig0 = min(10, max(0, np.log10(Sig0[ifrom[i]])))
        tmp2[i] = sp.interpolate.interp1d(np.log10(s_sig0), tmp1[:,i], kind='linear')(log10sig0)
    
    sigS = sp.sparse.coo_matrix((tmp2, (ifrom, ito)), shape=(ng, ng)).toarray()

    #return sigS
print(sigS.shape)

# Close the HDF5 files
hdf5_H01.close()
hdf5_O16.close()
hdf5_U235.close()
"""

'#def interpSigS(jLgn, s, Sig0):\nhdf5_H01  = h5py.File(micro_XS_path + \'/micro_H_001__294K.h5\', \'r\')\nhdf5_O16  = h5py.File(micro_XS_path + \'/micro_O_016__294K.h5\', \'r\')\nhdf5_U235 = h5py.File(micro_XS_path + \'/micro_U_235__294K.h5\', \'r\')\n\n\njLgn = 0\nSig0 = H2OU_sig0[1, :]\n\nelement = \'U235\'\n\n# number of energy groups\nng = 421\ns_sig0 = np.array(eval(f\'hdf5_{element}\').get(\'sig0_G\').get(\'sig0\'))\n\nfindSigS = list(eval(f\'hdf5_{element}\').get(group).items())\nprint(f\'Items in hdf5_{element}:\', findSigS)\nstring = findSigS[-1][0]\n\n# Extract numbers using regular expression pattern\npattern = r"sigS\\((\\d+),(\\d+)\\)"\nmatch = re.search(pattern, string)\n\nif match:\n    x_4D = int(match.group(1)) + 1\n    y_4D = int(match.group(2)) + 1\n    print(f"i = {x_4D}, j = {y_4D}")\nelse:\n    print("No match found.")\n\n# Create the empty 3D numpy array\ns_sigS = np.zeros((x_4D, y_4D, ng, ng))\n\n# Access the data from the subgroups and store it in the 3D array

In [31]:
"""
hdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')

def get_subgroup_data_as_array(group):
    subgroups_data = []

    def get_data(name, obj):
        if isinstance(obj, h5py.Dataset):
            subgroup_data = np.array(obj)
            subgroups_data.append(subgroup_data)

    group.visititems(get_data)
    return np.array(subgroups_data)[0]

sigT_G_test = hdf5_U235.get('sigT_G')
sigT_data = get_subgroup_data_as_array(sigT_G_test)

print(sigT_G_test)
print(sigT_data)
"""


"\nhdf5_U235 = h5py.File(micro_XS_path + '/micro_U_235__294K.h5', 'r')\n\ndef get_subgroup_data_as_array(group):\n    subgroups_data = []\n\n    def get_data(name, obj):\n        if isinstance(obj, h5py.Dataset):\n            subgroup_data = np.array(obj)\n            subgroups_data.append(subgroup_data)\n\n    group.visititems(get_data)\n    return np.array(subgroups_data)[0]\n\nsigT_G_test = hdf5_U235.get('sigT_G')\nsigT_data = get_subgroup_data_as_array(sigT_G_test)\n\nprint(sigT_G_test)\nprint(sigT_data)\n"